In [2]:
from scipy import integrate
import numpy as np

In [2]:
def indicator(x, intervals):
    '''
Define the indicator function
     indicator function is function which if x value is inside the bound, you will get 1
     Otherwise you will get 0
    '''
    '''
    Require:
    x, left_bound, right_bound must have the same dimension
    '''
    '''
    Parameters: 
    
        x: 1 x n vector representing the point to check (Time dimension should be excluded)

        intervals: 2d (n x 2) arrays. First dimension is all the  spatial dimensions, and second dimension are 
                left and right bound of the subdomain
    
    return: 
        1 or 0, should be clear enough
    
    '''
    if len(x) != len(len(intervals[:, 0])):
        raise ValueError("Parameter dimensions do not agree.")
        
    for i in np.arange(len(intervals[:, 0])):
        if x[i] < intervals[i, 0] or x > intervals[i, 1]:
            return 0
    return 1

In [3]:
def compute_integral(X, grid_ndim, num_x, j, endpts):
    '''
    Parameters: 
    
        X: data grid
        
        grid_ndim: number of spatial dimensions. this is already a param in pysindy
        
        num_x: number of spatial datapoints in the grid.
        
        j: feature index
        
        endpts: 2 x n array 
            the first column is the left endpoints of the subdomain's each of the n dimensions,
            second column is right endpoint of each of the subdomain's each of the n dimensions
            
    return:
        nd integral within a subdomain
    '''
# find weight here
    



#     All the 1D weights will be stored in a 2D matrix as cols
#     sudo_var1: max number of pts per dim.
    weights = []
    for i in np.arange(grid_ndim):
#         We actually need a slicer here
#         this_dim = grid points in the i-th spatial dimension
        weight = get_1D_weight(this_dim, endpts[:, i])
#         append is extremely slow, but since its only used grid_ndim times it shouldn't bottle neck.
        weights.append(weight)
    

    return sp.integral.nquad(get_u_j(x, num_x, t, j), endpts)

In [99]:
def get_1D_weight(grid, endpt):
    '''
    Parameters: 
        grid: an 1D array that contains the value of the corresponding dimension of each grid points.
        
        endpt: 
    '''
    
#     initialize a bunch of 0
    weight = np.zeros(len(grid))

#     find the index at which we enter Omega_k in this axis
    start = 0
    end = 0
    record_start = True
    record_end = True
    for i in np.arange(len(grid)):
        if (grid[i] >= endpt[0] and record_start == True):
            start = i
            record_start = False
        elif (grid[i] >= endpt[1] and record_end == True):
            end = i
            record_end = False
            
#     the weight of all other grid points is 0 as they contribute nothing to the integral
#     and each grid point in omega_k needs a weight

#     start and end index has different equation for weight, so we do those first
    weight[start] = 1/2*(grid[start+1]-grid[start])
    weight[end] = 1/2*(grid[end]-grid[end-1])
    for i in np.arange(end-start-1): 
        weight[start+i+1] = 1/2*(grid[start+i+2]-grid[start+i])
    
    return weight

In [4]:
def get_u_j(x, num_x, t, j):
    '''
    x: data value x', a constant after stacking.
    t: time point t
    j: the feature of u that will get returned.
    '''  
#     spatial-temporal stack index
    ind = x*num_x+t
    
#   X is 2D, 
    return X[ind, j]

In [5]:
def get_omega_bound(index, intervals):
    '''
    Parameter:
        index: index of the subdomain to get bound of
        intervals: boundary of each subdomain correspond to each dimension
        
    return:
        2d (n x 2) arrays. First dimension is all the  spatial dimensions, and second dimension are left and right bound of the subdomain
    '''
    
    return intervals[index]

In [6]:
def get_theta_nonloc(j, k, kprime, intervals):
#     get how many time points are there
    num_t = np.shape(spatiotemporal_grid)[-2]
#     get how many spatial points are there
    num_x = np.prod(np.shape(spatiotemporal_grid)[:-2])
    
    theta_nonloc_p = np.zeros(num_t*num_x)
    
    for i in np.arange(theta_nonloc_p.length):
        this_t = i % num_t
        this_x = int(i/num_t)
        
#       get x(all x, this_t)
#       This currently filters the spatial temporal grid, as opposed to X
#       We mat not need this though, as we can use num_t and num_x to figure out the stacking of X.
#         n_dims  = len(np.shape(spatiotemporal_grid))
#         s = [slice(None) for i in range(n_dims)]
#         s[-2] = this_t
#         grid_t = spatiotemporal_grid[tuple(s)]
        
        coefficient = indicator(this_x, get_omega_bound(k, intervals))
        
        integral = compute_integral(this_x, this_t, num_x, j, get_omega_bound(kprime, intervals), X)
        
        theta_nonloc_p[i] = coefficient * integral
        
    return theta_nonloc_p

In [7]:
def func(*args):
    return np.sum(args)

# Below is the session where we test each block.

In [105]:
# 1D weight test starts here. 
sample_grid = np.linspace(0, 15, 21, endpoint=True)
endpts = [1, 9]
print(sample_grid)
print(get_1D_weight(sample_grid, endpts))

[ 0.    0.75  1.5   2.25  3.    3.75  4.5   5.25  6.    6.75  7.5   8.25
  9.    9.75 10.5  11.25 12.   12.75 13.5  14.25 15.  ]
[0.    0.    0.375 0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75  0.75
 0.375 0.    0.    0.    0.    0.    0.    0.    0.   ]


# The above cell shows that the 1D weight calculation is performing properly.